Keras Tuner : tensorflow에 대한 최적의 하이퍼파라미터 세트를 선택하는데 도움을 주는 라이브러리

### Hyperparameter의 유형
1. 숨겨진 레이어의 수 및 너비와 같이 모델 선택에 영향을 미치는 모델 하이퍼파라미터
2. SGD의 학습률 및 KNN 분류자의 최근접 이웃수와 같은 학습 알고리즘의 속도와 품질에 영향을 주는 알고리즘 하이퍼파라미터

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [3]:
!pip install -q -U keras-tuner

import kerastuner as kt

     |████████████████████████████████| 97 kB 3.2 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# fashion mnist 데이터 로드

(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### 모델 정의하기

하이퍼튜닝을 위한 모델을 빌드할 때는 모델 아키텍처와 더불어 하이퍼파라미터 검색 공간도 정의

하이퍼 모델 : 하이퍼튜닝을 위해 설정하는 모델

하이퍼모델을 정의하는 접근 방식
1. 모델 빌더 함수 사용
 - 컴파일된 모델을 반환하고 인라인으로 정의한 하이퍼파라미터를 사용하여 모델을 하이퍼튜닝
2. keras tuner API의 HyperModel 클래스를 하위 클래스화

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape = (28, 28)))

  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])

  return model

튜너 인스턴트화, 하이퍼튜닝 수행하기

In [8]:
tuner = kt.Hyperband(model_builder, objective = 'val_accuracy', max_epochs = 10,
                     factor = 3, directory = 'my_dir', project_name = 'intro_to_kt')

In [9]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])


In [12]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.001.



In [13]:
# 최적의 파라미터로 훈련

model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4899 - accuracy: 0.8281 - val_loss: 0.4162 - val_accuracy: 0.8550
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3696 - accuracy: 0.8658 - val_loss: 0.4276 - val_accuracy: 0.8504
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3315 - accuracy: 0.8783 - val_loss: 0.3556 - val_accuracy: 0.8750
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3099 - accuracy: 0.8869 - val_loss: 0.3641 - val_accuracy: 0.8705
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2921 - accuracy: 0.8916 - val_loss: 0.3572 - val_accuracy: 0.8738
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2762 - accuracy: 0.8988 - val_loss: 0.3538 - val_accuracy: 0.8740
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2662 - accuracy: 0.9016 - val_loss: 0.3276 - val_accuracy: